In [1]:
import numpy as np
import rpy2
import rpy2.robjects as robjects
import pickle
from time import time

In [2]:
def store_results_dict(results, name):
    pickle.dump(results, open("./Results/{}.p".format(name), "wb" ))
    
def retrieve_results_dict(name):
    try:
        return pickle.load(open( "./Results/{}.p".format(name), "rb" ))
    except Exception as e:
        return None

### Data Generation

#### Raw Data

In [3]:
# GLOBAL CONFIG

# Var count
n_vars = 10

# Data types (default is standard normal)
binary_indeces = [1, 3, 6, 8, 9]
binarize = True

# Associations between vars an treat/outcome
treat_vars = [0,1,2,3,4,5,6,7]
outcome_vars = [0,1,2,3,4,8,9,10]

# Treat/outcome generation weights
assignment_weights = np.array([0, 0.8, -0.25, 0.6, -0.4, -0.8, -0.5, 0.7])
outcome_weights = np.array([-3.85, 0.3, -0.36, -0.73, -0.2, 0.71, -0.19, 0.26])
true_treat_effect = -0.4

def generate_data(n_samples=1000):
    # Generate 10 Random Vars
    # 1-4 are confounders: associated with outcome + treatment
    # 5-7 are exposure predictors
    # 8-10 are outcome predictors
    X = np.random.normal(loc=0.0, scale=1.0, size=(n_samples, n_vars))

    # Binarize specified vars if requested.
    if binarize:
        for var in binary_indeces:
            X[:, var-1] = (X[:, var -1] > 0).astype(int)

    # Add dummy for bias param     
    X = np.hstack([np.ones((n_samples, 1)), X])
    return X

In [4]:
# DEBUG
# X = generate_data(2000)
# X.shape

#### Assignment

In [5]:
# Create the models

assignment_models={}

def nonlinear_transform(X, B, quad_indeces):
    for quad_index in quad_indeces:
        quad = X[:, quad_index]**2
        X = np.hstack([X, quad.reshape(-1, 1)])
        B = np.append(B, B[quad_index])
    
    return X, B

def nonadditive_transform(X, B, interaction_indeces, interaction_weights=None):
    for interaction_index, var_indeces in enumerate(interaction_indeces):
        int_1, int_2 = var_indeces
        interaction_val = X[:, int_1]*X[:, int_2]
        
        if not interaction_weights:
            interaction_val = interaction_val*0.5
        else:
            interaction_val = interaction_val*interaction_weights[interaction_index]
            
        X = np.hstack([X, interaction_val.reshape(-1, 1)])
        B = np.append(B, B[int_1])
    
    return X, B

# Scenario 1
assignment_models["A_add_lin"] = lambda B, X: np.dot(X, B)

# Scenario 2:     
assignment_models["B_add_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2]))
# Scenario 3:
assignment_models["C_add_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2, 4, 7]))
# Scenario 4:
assignment_models["D_mild_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]))

# Scenario 5:
assignment_models["E_mild_nadd_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]), quad_indeces=[2]))
# Scenario 6
assignment_models["F_mod_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]))
# Scenario 7
assignment_models["G_mod_nadd_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]), 
                                                                            quad_indeces=[2, 4, 7]))

In [6]:
# Tests 
assert(set(assignment_models["A_add_lin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([9, 37]))

assert(set(assignment_models["B_add_mild_nlin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([33, 637]))

assert(set(assignment_models["C_add_mod_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([373, 13457]))

assert(set(assignment_models["D_mild_nadd_lin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([139.5, 3632]))

assert(set(assignment_models["E_mild_nadd_mild_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([163.5, 4232]))


#### Outcome and Assignment Functions

In [7]:
def get_assignments(B, X, n_samples, scenario="A_add_lin"):
    X_usable = X[:, treat_vars]
    
    # Calculate the probabilities of assignment
    linear_assignment_data = assignment_models[scenario](B, X_usable)
    p_treat = 1.0/(1+np.exp(-1*linear_assignment_data))

    # Assign
    rand = np.random.random(n_samples)
    assignments = (rand < p_treat).astype(int)
    
    return assignments

In [8]:
def get_outcomes(B, X, assignments, effect=true_treat_effect):
    X_usable = X[:, outcome_vars]
    return effect*assignments + np.dot(X_usable, B)

In [9]:
# DEBUG
# assignments = get_assignments(assignment_weights, X, "mild_nonaddititive_mild_nonlinear")
# outcomes = get_outcomes(outcome_weights, X, assignments)

### Analysis

In [10]:
from rpy2.robjects import IntVector, FloatVector, Formula
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()

stats = importr('stats')
matching = importr('Matching')
snow = importr('snow')

Don't forget to port forward!:

```
# ~/.bash_profile
# Allow remote host to connect to local machine
# usage: $ remote_pfwd hostname {6000..6009}
function remote_pfwd {
  for i in ${@:2}
  do
    ssh -N -R $i:localhost:$i $1 &
  done
}
```
`remote_pfwd ubuntu@52.90.20.45 {11305..11307}`

In [11]:
class ClusterProvider(object):
    def __init__(self, n_nodes=8, remote_host=None):
        if remote_host is None:
            self.cl = snow.makeSOCKcluster(["localhost"]*n_nodes)
        else:
            port = 11305
            self.cl = snow.makeSOCKcluster([remote_host]*n_nodes, rscript="Rscript", manual=True, snowlib="/usr/local/lib/R/site-library",
                                           port=IntVector(list(range(port, port+n_nodes))), master="localhost", outfile="/dev/stdout")
    
    def get_cluster(self):
        return self.cl
    
    def kill_cluster(self):
        snow.stopCluster(self.cl)

In [23]:
cluster_provider = ClusterProvider(n_nodes=4, remote_host="ubuntu@184.73.51.120")

In [22]:
cluster_provider.kill_cluster()

#### Estimators

Define methods which can process outcomes, assignments and covariate data into a treatment effect estimate. 

1. Logistic Regression
2. GenMatch
3. VAE

In [14]:
# 1. Logisic Regression Propensity Matching
def logistic_prop_matching_est(outcomes, assignments, covariate_data):
    # Setup
    y = IntVector(assignments)
    fmla = Formula('y ~ X')
    env = fmla.environment
    
    # Run propensiy regression
    env['X'] = covariate_data
    env['y'] = y
    fit = stats.glm(fmla, family="binomial")
    
    # DEBUG: fit.rx("coefficients")
    propensity_scores = fit.rx2("fitted.values")
    
    # Run matching
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=propensity_scores,
        replace=True)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [15]:
# 2. GenMatch Matching
def genmatch_est(outcomes, assignments, covariate_data):
    
    cl = cluster_provider.get_cluster()
    
    start = time()
    gen_out = matching.GenMatch(
        Tr=IntVector(assignments),
        X=covariate_data,
        # pop_size=10, max_generations=10, wait_generations=1, (use default params)
        print_level=0,
        cluster=cl)
    print("GenMatch Time: ", time() - start)
    
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=covariate_data,
        replace=True,
        Weight_matrix=gen_out)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [16]:
# DEBUG
# est = logistic_prop_matching_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

In [17]:
# DEBUG
# est = genmatch_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

#### Monte Carlo Runner Code

In [18]:
def get_data(n_samples, assignment_model):
    X = generate_data(n_samples)
    assignments = get_assignments(assignment_weights, X,
                                  n_samples, assignment_model)

    outcomes = get_outcomes(outcome_weights, X, assignments)
    
    return assignments, outcomes, X

def get_estimate(outcomes, assignments, covar_data, method):
    return method(outcomes, assignments, covar_data)

In [19]:
def run_simulation(runs=1000, n_samples=1000,
                   assignment_model="additive_linear",
                   estimator=logistic_prop_matching_est,
                   verbose=True):
    
    progress_tick = max(1, int(runs/10))
    results = np.zeros(runs)

    for i in range(runs):
        assignments, outcomes, covar_data = get_data(n_samples, assignment_model)
        
        covar_data = covar_data[:, 1:] #exclude bias term
        results[i] = get_estimate(outcomes, assignments,
                                  covar_data, estimator)
        
        if i%progress_tick == progress_tick-1 and verbose:
            print("Done {} of {}".format(i+1, runs))
    
    bias = np.abs(np.mean((true_treat_effect-results)/true_treat_effect)*100)
    rmse = np.mean((true_treat_effect-results)**2)**0.5
    
    if verbose:
        print("\nRMSE", rmse)
        print("Bias", bias)
        print("===============\n\n")
    
    return {"RMSE": rmse, "Bias": bias}

In [24]:
# DEBUG:
run_simulation(runs=5, n_samples=1000, assignment_model="A_add_lin",
              estimator=genmatch_est)

/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: rgenoud

  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: parallel

  warnings.warn(x, RRuntimeWarning)


GenMatch Time:  1062.8477492332458
Done 1 of 5


/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Warning messages:

  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 1: 
  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: In (function (Tr, X, BalanceMatrix = X, estimand = "ATT", M = 1,  :
  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
 
  warnings.warn(x, RRuntimeWarning)
/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning:  The key tuning parameters for optimization were are all left at their default values.  The 'pop.size' option in particular should probably be increased for optimal results.  For details ple

RRuntimeError: 

### Run MC trial

In [ ]:
def run_test_battery(est,
                     store_name=None, 
                     runs=1000,
                     n_samples=1000,
                     overwrite=False, verbosity=1):
    # Logging
    def printer(level, *args):
        if level <= verbosity:
            print(*args)
    
    # Storage
    if store_name is None:
        store_name = "est_{}_runs_{}_n_{}".format(
            est.__name__,
            runs,
            n_samples)
    
    results = retrieve_results_dict(store_name)

    if overwrite or (not results):
        printer(1, "No valid, existant results found. Beggining battery.\n")
        results = {}
        for model in assignment_models.keys():
            printer(1, "Running: ", model)
            results[model] = run_simulation(
                                runs=runs,
                                n_samples=n_samples,
                                assignment_model=model,
                                estimator=est,
                                verbose=(verbosity==2))
            printer(1, "Done.\n")

        store_results_dict(results, store_name)
    else:
        printer(1, "Displaying cached results.\n")
    
    printer(1, "Results")
    for model, results in results.items():
        printer(1, "Model: ", model)
        print(1, "Bias: ", results["Bias"])
        print(1, "RMSE: ", results["RMSE"], "\n")

In [ ]:
run_test_battery(
    est=logistic_prop_matching_est,
    runs=1000,
    n_samples=1000)